In [1]:
import requests
import re
import json
import os
import pandas as pd
import sys
import numpy as np

# project lib
PROJECT_SRC_PATH = os.path.join( '/workspace/workspace/ufo-prediction', 'src-RCA-UFO')
sys.path.append(PROJECT_SRC_PATH)
import dataset

In [2]:
# Set a seed for reproducibility
np.random.seed(42)  # You can choose any number as your seed

# Define paths for data
path_data_NLD = os.path.join('/workspace/workspace/ufo-prediction', 'demo', 'df-NLD.pkl')
path_data_FRA = os.path.join('/workspace/workspace/ufo-prediction', 'demo', 'df-FRA.pkl')
path_data_ESP = os.path.join('/workspace/workspace/ufo-prediction', 'demo', 'df-ESP.pkl')
processed_df_NLD = "/workspace/workspace/ufo-prediction/demo/kartaview_key_NLD_adv.csv"
processed_df_ESP = "/workspace/workspace/ufo-prediction/demo/kartaview_key_ESP_adv.csv"
processed_df_FRA = "/workspace/workspace/ufo-prediction/demo/kartaview_key_FRA_adv.csv"
processed_df_ALL = "/workspace/workspace/ufo-prediction/demo/kartaview_key.csv"

# Image directory paths
image_dir_map = {
    'NLD': '/workspace/workspace/ufo-prediction/image_data_NLD_adv',
    'FRA': '/workspace/workspace/ufo-prediction/image_data_FRA_adv',
    'ESP': '/workspace/workspace/ufo-prediction/image_data_ESP_adv',
    'ALL': '/workspace/workspace/ufo-prediction/image_data'
}

# Ask for user input
country_code = input("Enter country code (FRA, NLD, ESP) or ALL: ").upper()

# Map user input to the correct path
path_data_map = {
    'NLD': processed_df_NLD,
    'FRA': processed_df_FRA,
    'ESP': processed_df_ESP,
    'ALL': processed_df_ALL
}

raw_data_path_map = {
'NLD': path_data_NLD,
'FRA': path_data_FRA,
'ESP': path_data_ESP
}
# Check if the input is valid
if country_code not in path_data_map:
    print("Invalid country code or specification. Please enter FRA, NLD, ESP, or ALL.")
else:
    processed_df_path = path_data_map[country_code]
    # Set directory based on country code
    current_directory = image_dir_map[country_code]

    # Check if the new directory exists, if not, create it
    if not os.path.exists(current_directory):
        os.makedirs(current_directory)
        print(f"Directory {current_directory} created.")
    else:
        print(f"Directory {current_directory} already exists.")

    # Process for ALL
    if country_code == 'ALL':
        if os.path.exists(processed_df_path):
            kartaview_keys = pd.read_csv(processed_df_path)
            print("Loaded processed DataFrame from", processed_df_path)
        else:
            print("Creating a new processed DataFrame for ALL")
            path_data_RCA = os.path.join(dataset.DATA_DIR, 'rca-ufo-merge_ALL.csv')
            df = pd.read_csv(path_data_RCA, encoding='latin1')
            kartaview_keys = df[['lon', 'lat','age_right', 'id', 'PropertyKey_ID']]
            kartaview_keys.to_csv(processed_df_path, index=False)
            

    # Process for FRA, NLD, ESP
    else:
        if os.path.exists(processed_df_path):
            kartaview_keys = pd.read_csv(processed_df_path)
            print("Loaded processed DataFrame from", processed_df_path)
        else:
            print(f"Creating a new processed DataFrame for {country_code}")
            df_path = raw_data_path_map[country_code]
            df = pd.read_pickle(df_path)
            print("Loaded DataFrame from", df_path)
            sampled_df = df.sample(n=1500000, random_state=42)
            kartaview_keys = sampled_df[['lon', 'lat', 'age', 'id']].rename(columns={'age': 'age_right'})
            kartaview_keys.to_csv(processed_df_path, index=False)

print(kartaview_keys) 

Enter country code (FRA, NLD, ESP) or ALL: ESP
Directory /workspace/workspace/ufo-prediction/image_data_ESP_adv already exists.
Loaded processed DataFrame from /workspace/workspace/ufo-prediction/demo/kartaview_key_ESP_adv.csv
             lon        lat  age_right                          id
0      -0.566147  39.367118     2007.0  v0.1-ESP.10.3.13.25_1-8727
1      -7.313844  43.344895     1975.0    v0.1-ESP.12.2.3.1_1-3452
2      -5.612333  40.487875     1930.0      v0.1-ESP.5.5.9.7_1-394
3      -4.207111  43.003097     1920.0      v0.1-ESP.3.1.3.3_1-980
4      -3.560018  43.455583     2000.0   v0.1-ESP.3.1.11.11_1-1148
...          ...        ...        ...                         ...
908309 -5.984130  37.675687     2011.0     v0.1-ESP.1.8.1.4_1-5558
908310  2.505601  41.879092     1900.0     v0.1-ESP.6.2.6.19_1-823
908311 -3.924419  37.463344     1980.0    v0.1-ESP.1.6.9.1_1-10785
908312 -0.220692  38.513329     1945.0   v0.1-ESP.10.1.7.17_1-5766
908313 -4.000000        NaN        N

In [3]:
print(current_directory)
print(kartaview_keys)

/workspace/workspace/ufo-prediction/image_data_ESP_adv
             lon        lat  age_right                          id
0      -0.566147  39.367118     2007.0  v0.1-ESP.10.3.13.25_1-8727
1      -7.313844  43.344895     1975.0    v0.1-ESP.12.2.3.1_1-3452
2      -5.612333  40.487875     1930.0      v0.1-ESP.5.5.9.7_1-394
3      -4.207111  43.003097     1920.0      v0.1-ESP.3.1.3.3_1-980
4      -3.560018  43.455583     2000.0   v0.1-ESP.3.1.11.11_1-1148
...          ...        ...        ...                         ...
908309 -5.984130  37.675687     2011.0     v0.1-ESP.1.8.1.4_1-5558
908310  2.505601  41.879092     1900.0     v0.1-ESP.6.2.6.19_1-823
908311 -3.924419  37.463344     1980.0    v0.1-ESP.1.6.9.1_1-10785
908312 -0.220692  38.513329     1945.0   v0.1-ESP.10.1.7.17_1-5766
908313 -4.000000        NaN        NaN                         NaN

[908314 rows x 4 columns]


In [ ]:
# Assuming current_directory and processed_df_path are set from the previous code chunk
print(f"Images will be saved in: {current_directory}")
print(f"Using kartaview_keys from: {processed_df_path}")

# Consistency check between image directory and kartaview_keys path
expected_csv_map = {
    '/workspace/workspace/ufo-prediction/image_data': '/workspace/workspace/ufo-prediction/demo/kartaview_key.csv',
    '/workspace/workspace/ufo-prediction/image_data_NLD': '/workspace/workspace/ufo-prediction/demo/kartaview_key_NLD.csv',
    '/workspace/workspace/ufo-prediction/image_data_FRA': '/workspace/workspace/ufo-prediction/demo/kartaview_key_FRA.csv',
    '/workspace/workspace/ufo-prediction/image_data_ESP': '/workspace/workspace/ufo-prediction/demo/kartaview_key_ESP.csv',
}

# Stop the code if using image_data directory
if current_directory == '/workspace/workspace/ufo-prediction/image_data':
    print("Download for the 'image_data' directory has already been completed. Stopping execution.")
    # Use `exit()` or `sys.exit()` depending on your environment
    exit()

if processed_df_path != expected_csv_map.get(current_directory):
    print("Inconsistency detected between the image directory and the kartaview_keys path. Please check.")
    exit()

image_count = {} 
print("Number of buildings remaining: ",len(kartaview_keys))

#Initialise a counter for the loop iterations
iteration_counter = 0

for index, row in kartaview_keys.iterrows():
    iteration_counter += 1 # Increment the counter with each iteration
    precision = 6  # Start with 6 decimal places
    success = False  # Flag to indicate if the request was successful

    while precision > 2 and not success:
        # Format lon and lat to the current precision
        lon = f"{row['lon']:.{precision}f}"
        lat = f"{row['lat']:.{precision}f}"

        # Construct the API URL
        url = "https://api.openstreetcam.org/2.0/photo/?lat={}&lng={}".format(lat, lon)

        # Send a GET request to the API
        response = requests.get(url)

        # Check if the request was successful
        if response.status_code == 200:
            data = response.json()
            url_pattern = r'https://storage\d+\.openstreetcam\.org/files/photo/\d+/\d+/\d+/[^"]+\.jpg'
            urls = re.findall(url_pattern, json.dumps(data))

            filtered_urls = [
                url for url in urls
                if not any(x in url.rsplit('/', 2)[-2] for x in ["{{sizeprefix}}", "proc"]) and
                ("th" in url.rsplit('/', 2)[-2] and not "lth" in url.rsplit('/', 2)[-2])
            ]

            if filtered_urls:
                # Initialize or update the image count for the current ID
                building_id = row['id']  # Assuming 'id' column exists in your DataFrame
                if building_id not in image_count:
                    image_count[building_id] = 0

                for image_url in filtered_urls:
                    image_count[building_id] += 1  # Increment the image count for the building
                    subscript = image_count[building_id]  # Subscript for the file name
                    file_name = f"{row['age_right']}_{building_id}_{subscript}.jpg"
                    file_path = os.path.join(current_directory, file_name)

                    # Check if the file already exists
                    if os.path.exists(file_path):
                        print(f"File already exists: {file_path}. Skipping download.")
                    else:
                        image_response = requests.get(image_url)

                        if image_response.status_code == 200:
                            with open(file_path, 'wb') as f:
                                f.write(image_response.content)
                            print("Image downloaded successfully: {}".format(file_path))
                        else:
                            print("Failed to download the image.")
                success = True  # Mark success as True to exit the while loop
            else:
                print("No suitable images found for location: lon={}, lat={}".format(lon, lat))
                precision -= 1  # Reduce precision by one decimal place
        else:
            print("Failed to retrieve data from the API for location: lon={}, lat={}. Trying with reduced precision.")
            

    # After processing, remove the row from df_subset
    kartaview_keys = kartaview_keys.drop(index)

    # Only save the updated DataFrame to a CSV file every 100th instance
    if iteration_counter % 100 == 0:
        print(f"Saving progress at iteration {iteration_counter} to {processed_df_path}. ")
        print("Number of buildings remaining: ",len(kartaview_keys))
        kartaview_keys.to_csv(processed_df_path, index=False)

    if not success:
        print("Unable to retrieve data from the API with sufficient precision for location: lon={}, lat={}".format(row['lon'], row['lat']))
        
if iteration_counter % 100 != 0:
    print(f"Saving final progress")
    print("Number of buildings remaining: ",len(kartaview_keys))
    kartaview_keys.to_csv(processed_df_path, index=False)

Images will be saved in: /workspace/workspace/ufo-prediction/image_data_ESP_adv
Using kartaview_keys from: /workspace/workspace/ufo-prediction/demo/kartaview_key_ESP_adv.csv
Inconsistency detected between the image directory and the kartaview_keys path. Please check.
Number of buildings remaining:  908314
No suitable images found for location: lon=-0.566147, lat=39.367118
No suitable images found for location: lon=-0.56615, lat=39.36712
No suitable images found for location: lon=-0.5661, lat=39.3671
No suitable images found for location: lon=-0.566, lat=39.367
Unable to retrieve data from the API with sufficient precision for location: lon=-0.5661473176409938, lat=39.36711813596284
No suitable images found for location: lon=-7.313844, lat=43.344895
No suitable images found for location: lon=-7.31384, lat=43.34489
No suitable images found for location: lon=-7.3138, lat=43.3449
No suitable images found for location: lon=-7.314, lat=43.345
Unable to retrieve data from the API with suffici

No suitable images found for location: lon=1.5507, lat=41.2464
No suitable images found for location: lon=1.551, lat=41.246
Unable to retrieve data from the API with sufficient precision for location: lon=1.5507258515262872, lat=41.24635694785032
No suitable images found for location: lon=-1.895892, lat=41.663676
No suitable images found for location: lon=-1.89589, lat=41.66368
No suitable images found for location: lon=-1.8959, lat=41.6637
No suitable images found for location: lon=-1.896, lat=41.664
Unable to retrieve data from the API with sufficient precision for location: lon=-1.8958918357093184, lat=41.66367626556164
No suitable images found for location: lon=3.197667, lat=41.896045
No suitable images found for location: lon=3.19767, lat=41.89605
No suitable images found for location: lon=3.1977, lat=41.8960
No suitable images found for location: lon=3.198, lat=41.896
Unable to retrieve data from the API with sufficient precision for location: lon=3.1976665424737063, lat=41.89604

No suitable images found for location: lon=-3.959901, lat=37.769650
No suitable images found for location: lon=-3.95990, lat=37.76965
No suitable images found for location: lon=-3.9599, lat=37.7697
No suitable images found for location: lon=-3.960, lat=37.770
Unable to retrieve data from the API with sufficient precision for location: lon=-3.959901136156625, lat=37.76965026405405
No suitable images found for location: lon=4.300718, lat=39.887324
No suitable images found for location: lon=4.30072, lat=39.88732
No suitable images found for location: lon=4.3007, lat=39.8873
No suitable images found for location: lon=4.301, lat=39.887
Unable to retrieve data from the API with sufficient precision for location: lon=4.3007183607773385, lat=39.88732375781452
No suitable images found for location: lon=-0.624149, lat=38.805430
No suitable images found for location: lon=-0.62415, lat=38.80543
No suitable images found for location: lon=-0.6241, lat=38.8054
No suitable images found for location: l

No suitable images found for location: lon=-0.569, lat=39.543
Unable to retrieve data from the API with sufficient precision for location: lon=-0.568955943613483, lat=39.54297972425773
No suitable images found for location: lon=3.018798, lat=39.881685
No suitable images found for location: lon=3.01880, lat=39.88169
No suitable images found for location: lon=3.0188, lat=39.8817
No suitable images found for location: lon=3.019, lat=39.882
Unable to retrieve data from the API with sufficient precision for location: lon=3.018798358828433, lat=39.881685447673576
No suitable images found for location: lon=-2.695642, lat=38.694579
No suitable images found for location: lon=-2.69564, lat=38.69458
No suitable images found for location: lon=-2.6956, lat=38.6946
No suitable images found for location: lon=-2.696, lat=38.695
Unable to retrieve data from the API with sufficient precision for location: lon=-2.6956421196701887, lat=38.69457862544553
No suitable images found for location: lon=-2.886023

Image downloaded successfully: /workspace/workspace/ufo-prediction/image_data_ESP_adv/2008.0_v0.1-ESP.1.7.3.21_1-7887_1.jpg
Image downloaded successfully: /workspace/workspace/ufo-prediction/image_data_ESP_adv/2008.0_v0.1-ESP.1.7.3.21_1-7887_2.jpg
No suitable images found for location: lon=-6.397864, lat=41.316635
No suitable images found for location: lon=-6.39786, lat=41.31663
No suitable images found for location: lon=-6.3979, lat=41.3166
No suitable images found for location: lon=-6.398, lat=41.317
Unable to retrieve data from the API with sufficient precision for location: lon=-6.397863868303985, lat=41.31663470801144
No suitable images found for location: lon=0.205947, lat=38.738343
No suitable images found for location: lon=0.20595, lat=38.73834
No suitable images found for location: lon=0.2059, lat=38.7383
No suitable images found for location: lon=0.206, lat=38.738
Unable to retrieve data from the API with sufficient precision for location: lon=0.2059473429885981, lat=38.73834

...

No suitable images found for location: lon=-4.808307, lat=37.913184
No suitable images found for location: lon=-4.80831, lat=37.91318
No suitable images found for location: lon=-4.8083, lat=37.9132
No suitable images found for location: lon=-4.808, lat=37.913
Unable to retrieve data from the API with sufficient precision for location: lon=-4.80830716522098, lat=37.9131839271408
No suitable images found for location: lon=-5.077511, lat=40.214806
No suitable images found for location: lon=-5.07751, lat=40.21481
No suitable images found for location: lon=-5.0775, lat=40.2148
No suitable images found for location: lon=-5.078, lat=40.215
Unable to retrieve data from the API with sufficient precision for location: lon=-5.077511489720199, lat=40.21480579571067
No suitable images found for location: lon=-4.383735, lat=37.896224
No suitable images found for location: lon=-4.38373, lat=37.89622
No suitable images found for location: lon=-4.3837, lat=37.8962
No suitable images found for location:

No suitable images found for location: lon=-8.877, lat=41.904
Unable to retrieve data from the API with sufficient precision for location: lon=-8.877366371375365, lat=41.90403790284131
No suitable images found for location: lon=-3.208838, lat=38.249695
No suitable images found for location: lon=-3.20884, lat=38.24969
No suitable images found for location: lon=-3.2088, lat=38.2497
No suitable images found for location: lon=-3.209, lat=38.250
Unable to retrieve data from the API with sufficient precision for location: lon=-3.208838266084354, lat=38.24969457172481
No suitable images found for location: lon=-3.686698, lat=43.417628
No suitable images found for location: lon=-3.68670, lat=43.41763
No suitable images found for location: lon=-3.6867, lat=43.4176
No suitable images found for location: lon=-3.687, lat=43.418
Unable to retrieve data from the API with sufficient precision for location: lon=-3.686697587585473, lat=43.417627943674844
No suitable images found for location: lon=2.128

No suitable images found for location: lon=-0.537248, lat=39.526797
No suitable images found for location: lon=-0.53725, lat=39.52680
No suitable images found for location: lon=-0.5372, lat=39.5268
No suitable images found for location: lon=-0.537, lat=39.527
Unable to retrieve data from the API with sufficient precision for location: lon=-0.5372484456341884, lat=39.52679722863238
No suitable images found for location: lon=-3.603045, lat=39.071440
No suitable images found for location: lon=-3.60305, lat=39.07144
No suitable images found for location: lon=-3.6030, lat=39.0714
No suitable images found for location: lon=-3.603, lat=39.071
Unable to retrieve data from the API with sufficient precision for location: lon=-3.603045353819945, lat=39.0714404775125
No suitable images found for location: lon=-3.479792, lat=40.315842
No suitable images found for location: lon=-3.47979, lat=40.31584
No suitable images found for location: lon=-3.4798, lat=40.3158
No suitable images found for locatio

No suitable images found for location: lon=-0.605, lat=39.293
Unable to retrieve data from the API with sufficient precision for location: lon=-0.6050284156002811, lat=39.29331616299103
No suitable images found for location: lon=-2.164349, lat=39.209668
No suitable images found for location: lon=-2.16435, lat=39.20967
No suitable images found for location: lon=-2.1643, lat=39.2097
No suitable images found for location: lon=-2.164, lat=39.210
Unable to retrieve data from the API with sufficient precision for location: lon=-2.1643489959628592, lat=39.20966763269195
No suitable images found for location: lon=-3.337528, lat=40.645138
No suitable images found for location: lon=-3.33753, lat=40.64514
No suitable images found for location: lon=-3.3375, lat=40.6451
No suitable images found for location: lon=-3.338, lat=40.645
Unable to retrieve data from the API with sufficient precision for location: lon=-3.337527632466543, lat=40.64513751425453
No suitable images found for location: lon=3.06

No suitable images found for location: lon=-5.646019, lat=37.524578
No suitable images found for location: lon=-5.64602, lat=37.52458
No suitable images found for location: lon=-5.6460, lat=37.5246
No suitable images found for location: lon=-5.646, lat=37.525
Unable to retrieve data from the API with sufficient precision for location: lon=-5.646018893134404, lat=37.52457760644719
No suitable images found for location: lon=-4.176188, lat=38.709042
No suitable images found for location: lon=-4.17619, lat=38.70904
No suitable images found for location: lon=-4.1762, lat=38.7090
No suitable images found for location: lon=-4.176, lat=38.709
Unable to retrieve data from the API with sufficient precision for location: lon=-4.176188104188343, lat=38.70904164877886
No suitable images found for location: lon=4.244616, lat=39.839706
No suitable images found for location: lon=4.24462, lat=39.83971
No suitable images found for location: lon=4.2446, lat=39.8397
No suitable images found for location: 